## Quantum Approximate Optimisation Algorithm in Python

This notebook demostrates how the package in this repository can be used to solve quadratic unconstrained binary optimisation (QUBO) problems, expressed in the context of the max-cut problem from graph theory, using a quantum algorithm.

We start by first defining the graph using the $\texttt{networkx}$ package in Python. In this example we will be considering a quite simple complete graph with 4 vertices, where the weights of each edge are
uniformly distributed between 0 and 1.

In [ ]:
import networkx as nx
from src_code import get_data
import random

def generate_graph(n, seed=1):

    graph = nx.Graph()
    edge_list = []
    for n_1 in range(n):

        for n_2 in range(n_1+1, n):

            edge_list.append((n_1, n_2))
            
    graph.add_edges_from(edge_list)

    random.seed(seed)
    weights = [random.random() for i in range(len(edge_list))]

    for index, edge in enumerate(graph.edges()):
        graph.get_edge_data(*edge)['weight'] = weights[index]

    return graph, weights

In [ ]:
no_vertices = 4
graph = generate_graph(no_vertices)[0]
pos=nx.circular_layout(graph)
nx.draw_networkx(graph, pos)
labels = nx.get_edge_attributes(graph,'weight')
for edge in labels:
    labels[edge] = round(labels[edge], 3)
tmp = nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels)

Once we have generated the graph, we can define the Ising Hamiltonian whose maximum energy eigenstate corresponds to the solution to the problem. This is given by $$\hat{H}_C=-\frac{1}{4}\sum_{i,j=0}^{n-1}W_{ij}\hat{Z}_i\hat{Z}_j.$$ It should be noted that the eigenvalues of this Hamiltonian are not exactly equal to the values of their corresponding cuts. They differ by a constant term $A = \frac{1}{4}\sum_{i,j=0}^{n-1}W_{ij}$, which is removed as it doesn't influence to the operation of the algorithm. It is later added back in to generate correct results. Additionally, we can define all the commutators $$\left[\hat{H}_C,-i\hat{A}\right]\quad\forall \quad\hat{A}\in\mathcal{P}.$$ These are used to find the gradients in parameter space at each iteration which are used to determine which mixer operator from the pool $\mathcal{P}$ to be added to the ADAPT-QAOA circuit in the current layer.

In [ ]:
from src_code import build_operators

hamiltonian = build_operators.cut_hamiltonian(graph=graph)
gradient_ops_dict = build_operators.build_all_mixers(graph=graph)

In order to make the generation of the unitaries in the ansatz quicker between iterations of both the overall algorithm and the classical optimsation scheme, it is useful to pre-compute all the Pauli strings appearing in the exponents of the unitaries. This then allows one to use the identity $$e^{i\alpha\hat{P}}=\cos(\alpha)\hat{I}+i\sin(\alpha)\hat{P}$$ for single Pauli strings, to find the unitaries using simple floating point arithmetic, rather than matrix exponentiation during each iteration.

In [ ]:
pauli_ops_dict = build_operators.build_all_paulis(no_vertices)

All this pre-computation allows for significant speed-up in the execution of the algorithms. We now move on to actually running QAOA on the graph. We first perform the standard non-adaptive algorithm. To do so we require to pick a specific depth for the circuit, i.e., the number of layers it will contain. We set this to 5.

In [ ]:
circuit_depth = 5
qaoa_solution = get_data.run_standard_qaoa(graph, depth=circuit_depth, pauli_ops_dict=pauli_ops_dict)

In [ ]:
for key in qaoa_solution:
    print(key+':', qaoa_solution[key])

We now move on the ADAPT-QAOA. We use the same maximum depth of 5 as above.

In [ ]:
adapt_qaoa_solution = get_data.run_adapt_qaoa(graph, pauli_ops_dict, gradient_ops_dict, circuit_depth)

In [ ]:
for key in adapt_qaoa_solution:
    if key == 'all_mixers':
        continue
    print(key+':', adapt_qaoa_solution[key])

Finally, we solve the problem using the Dynamic ADAPT-QAOA which determines at each layer whether it is beneficial to the classical optimisation to include a Hamiltonian unitary or not. To do this, it is useful to generate a dictionary containing splits of each mixer operator into two operators, one which commutes with the Hamiltonian, and one which anti-commutes with it. This is possible for all mixers which are single Pauli strings.

In [ ]:
pauli_mixers_split_ops_dict = build_operators.split_all_mixers(graph)

In [ ]:
dynamic_adapt_qaoa_solution = get_data.run_dynamic_adapt_qaoa(graph, pauli_ops_dict, gradient_ops_dict, pauli_mixers_split_ops_dict, max_depth=circuit_depth)

In [ ]:
for key in dynamic_adapt_qaoa_solution:
    if key == 'all_mixers':
        continue
    print(key+':', dynamic_adapt_qaoa_solution[key])

Overall, we see that the three algorithm implementations converge to a good approximation ratio, with the adaptive problem-tailored ones achieving better results. The dynamic algorithm converges quicker compared to the non-dynamic version.